<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Onehot_A" data-toc-modified-id="Onehot_A-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Onehot_A</a></span></li><li><span><a href="#合并" data-toc-modified-id="合并-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>合并</a></span></li></ul></div>

# Onehot_A

In [1]:
# coding: utf-8

# filelist: 
#         train:40428967,  
#     minitrain:4042898,  
# miniminitrain:404291,  
#    test_click:4577464
#  

#import os
#os.chdir('/media/zhou/0004DD1700005FE8/AI/00/project_2/')
#os.chdir('E:/AI/00/project_2')

class flags(object):
    def __init__(self, file_name, onehot_cat):
        self.file_name = file_name
        self.onehot_cat = onehot_cat
        self.data_dir = '../data/project_2/data/{0}/'.format(self.onehot_cat)
        self.output_dir = '../data/project_2/output/{0}/'.format(self.onehot_cat)
        self.model_dir = '../data/project_2/models/{0}/'.format(self.onehot_cat)

class params(object):
    def __init__(self, onehot_name):
        self.threshold = 10
        self.chunksize = 1e3
        self.num_trees = 50
        self.deep = 15
        self.split = '='
        # ['A_cat', 'A_hour', 'A_xgb', 
        #  'B_cat', 'C_his']
        self.onehot_name = onehot_name
        self.lr_C = 1


In [2]:

import numpy as np
import pandas as pd
import time
import gc
import os
import scipy.sparse as ss
#from  sklearn.cross_validation  import  train_test_split 
#import xgboost as xgb
#from sklearn.metrics import accuracy_score,log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,log_loss
from sklearn.externals import joblib
from sklearn.model_selection import GridSearchCV

In [20]:
def Pred(file_name, onehot_cat, onehot_name, is_LR=True, save_sub=True):
    
    FLAGS = flags(file_name, onehot_cat)
    PARAMS = params(onehot_name)  

    file_name = FLAGS.file_name
    onehot_name = PARAMS.onehot_name
    output_path = FLAGS.output_dir
    model_path = FLAGS.model_dir
    
    print('Load Data')
    X_train = ss.load_npz(output_path+'{0}_X_{1}_all.npz'.format(file_name, onehot_name), )

    y_train = ss.load_npz(output_path+'{0}_y_{1}_all.npz'.format(file_name, onehot_name), )
    y_train = y_train.toarray().astype(np.float32)[0]
    # 先转成np.array, 把数据类型转为np.float32(此时为2维数组shape(1,n)), 转为1-D np.arrar
    # 2维数组shape(m,n)适用于多分类问题, 在二分类中不适用
    if is_LR:
        C_dict = {'A_cat':0.1, 'A_cat_xgb':0.01, 'A_hour':0.01, 'B_cat':0.1, 'B_cat_xgb':0.01,  'A_his':0.01, }
        c=C_dict[onehot_name]
        #导入模型
        print('Load Model')
        lr = joblib.load(model_path+'LR_C{0}_{1}.sklearn'.format(c,onehot_name))
        #进行评价
        print('Predict')
        train_preds = lr.predict_proba(X_train)[:,1]
    else:
        xgtrain = xgb.DMatrix(X_train, label = y_train,)
        num_trees = 50
        deep = 9
        #导入模型
        print('Load Model')
        bst_train = xgb.Booster(model_file=model_path + 'tree{0}_deep{1}_{2}.xgboost'.format(num_trees, deep, onehot_name))
        #进行评价
        print('Predict')
        train_preds = bst_train.predict(xgtrain,)
        onehot_name = onehot_name+'xgboost'

    if save_sub:
        #合并test_id与predict
        submission = pd.read_csv(model_path + '{0}_id.csv'.format(file_name), dtype=np.uint64)
        submission.insert(loc=1, column='click', value=train_preds)

        print('to submission.csv')
        #保存预测好的文件
        submission.to_csv(model_path + 'submission{0}.csv'.format(onehot_name), index=False)
    else:
        return onehot_name, train_preds


In [6]:
file_name = ['test_click', 'test_click', 'test_click', 'test_click', 
             'test_click', 'test_click', 'test_click', 'test_click', ]
onehot_cat = ['Onehot_A', 'Onehot_A', 'Onehot_A', 'Onehot_B', 
              'Onehot_B', 'Onehot_A', 'Onehot_A', 'Onehot_B', ]
onehot_name = ['A_cat', 'A_cat_xgb', 'A_hour', 'B_cat', 
               'B_cat_xgb',  'A_his', 'A_cat', 'B_cat', ]
is_LR = [True, True, True, True, 
         True, True, False, False, ]
is_save = [True, True, True, True, 
           True, True, True, True, ]

In [42]:
for a,b,c,d,e in zip(file_name, onehot_cat, onehot_name, is_LR, is_save):
    Pred(a,b,c,d,e )

Load Data
Load Model
Predict
to submission.csv
Load Data
Load Model
Predict
to submission.csv
Load Data
Load Model
Predict
to submission.csv
Load Data
Load Model
Predict
to submission.csv
Load Data
Load Model
Predict
to submission.csv
Load Data
Load Model
Predict
to submission.csv
Load Data


MemoryError: 

In [8]:
import xgboost as xgb

In [9]:
Pred('test_click', 'Onehot_A', 'A_cat', False, True)

Load Data
Load Model
Predict
to submission.csv


In [10]:
Pred('test_click', 'Onehot_B', 'B_cat', False, True)

Load Data
Load Model
Predict
to submission.csv


In [21]:
FLAGS = flags('test_click', 'Onehot_A')
PARAMS = params('A_cat')  

file_name = FLAGS.file_name
onehot_name = PARAMS.onehot_name
output_path = FLAGS.output_dir
model_path = FLAGS.model_dir

y_train = ss.load_npz(output_path+'{0}_y_{1}_all.npz'.format(file_name, onehot_name), )
y_train = y_train.toarray().astype(np.float32)[0]
merge_all = pd.DataFrame(index=np.arange(len(y_train)), columns=['click'], data=y_train)

In [22]:
file_name = ['test_click', 'test_click', 'test_click', 'test_click', 
             'test_click', 'test_click', 'test_click', 'test_click', ]
onehot_cat = ['Onehot_A', 'Onehot_A', 'Onehot_A', 'Onehot_B', 
              'Onehot_B', 'Onehot_A', 'Onehot_A', 'Onehot_B', ]
onehot_name = ['A_cat', 'A_cat_xgb', 'A_hour', 'B_cat', 
               'B_cat_xgb',  'A_his', 'A_cat', 'B_cat', ]
is_LR = [True, True, True, True, 
         True, True, False, False, ]
is_save = [False, False, False, False, 
           False, False, False, False, ]

In [23]:
for a,b,c,d,e in zip(file_name, onehot_cat, onehot_name, is_LR, is_save):
    onehot_name, train_preds = Pred(a,b,c,d,e )
    gc.collect()
    merge_all.insert(loc=0, column=onehot_name, value=train_preds)

Load Data
Load Model
Predict
Load Data
Load Model
Predict
Load Data
Load Model
Predict
Load Data
Load Model
Predict
Load Data
Load Model
Predict
Load Data
Load Model
Predict
Load Data
Load Model
Predict
Load Data
Load Model
Predict


# 合并

In [31]:
merge_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4577464 entries, 0 to 4577463
Data columns (total 9 columns):
B_cat_xgboost    float32
A_cat_xgboost    float32
A_his            float64
B_cat_xgb        float64
B_cat            float64
A_hour           float64
A_cat_xgb        float64
A_cat            float64
click            float32
dtypes: float32(3), float64(6)
memory usage: 296.8 MB


In [32]:
X_train = merge_all.iloc[:,:-1]
y_train = merge_all.iloc[:,-1]

In [33]:
onehot_name = 'all'
c=1

#导入模型
lr = joblib.load(model_path+'LR_C{0}_{1}.sklearn'.format(c,onehot_name))
#lr = LogisticRegression(multi_class='ovr', penalty='l2', solver='sag', C=c, n_jobs=-1)

#开始训练
start_time = time.time()
print('predicting . . . ')
#lr.fit(X_train, y_train, )

#保存模型
#joblib.dump(lr, model_path+'LR_sklearn.model')

#进行评价
train_preds = lr.predict_proba(X_train)[:,1]
#train_predictions = np.around(train_preds)
print('cost time:{0}'.format(int(time.time() - start_time)))

#合并test_id与predict
submission = pd.read_csv(model_path + '{0}_id.csv'.format(file_name), dtype=np.uint64)
submission.insert(loc=1, column='click', value=train_preds)

print('to submission.csv')
#保存预测好的文件
submission.to_csv(model_path + 'submission.csv', index=False)

predicting . . . 
cost time:1
to submission.csv


In [25]:
#保存模型
joblib.dump(lr, model_path+'LR_C{0}.sklearn_more{1}__'.format(c,threshold))

['../data/project_2/models/Onehot_A/LR_C0.1.sklearn_more10__']

In [ ]:
lr_model = joblib.load(model_path+model_path+'LR_C{0}.sklearn'.format(c))